# `Pipeline` в `sklearn`

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(font_scale=1.3)

В этом ноутбуке описывается класс `Pipeline`, встроенный в библиотеку `sklearn`. 

В библиотеке `sklearn` есть несколько категорий объектов:
* `Estimator` &mdash; объект, который имеет метод `fit`;
* `Predictor` &mdash; объект, который имеет метод `predict`;
* `Transformer` &mdash; объект, который имеет метод `transform`.

Более подробнее о структуре классов можно почитать по [ссылке](https://scikit-learn.org/stable/developers/develop.html).

Конкретный класс может относиться сразу к нескольким из этих категорий. Например, `Ridge` имеет методы `fit`, `predict`, `StandardScaler` методы `fit`, `transform`.

Инструмент `Pipeline` позволяет создать новый `Estimator` из цепочки последовательно применяющихся `Transformer` и `Estimator` в качестве последнего звена. Таким образом, мы можем зафиксировать различные преобразования над данными, которые надо выполнить перед обучением и применением модели.

При вызове метода `fit` для всех компонент кроме последней выполняется метод `fit_transform`, который обучает компоненту и передает трансформированные данные дальше по цепочке. Для последней компоненты выполняется только метод `fit`. 

При вызове методов `transform`, `predict` для всех компонент кроме последней вызывается метод `transform`, а для последней вызванный от всего пайплайна метод.

Более подробно об особенностях применения можно почитать по [ссылке](https://scikit-learn.org/stable/modules/compose.html).

## Данные
 
Датасет состоит из $11$ химических признаков вина. Откликом является переменная `quality` &mdash; качество вина.

In [2]:
winedf = pd.read_csv(
    'https://archive.ics.uci.edu/ml/'
    'machine-learning-databases/wine-quality/winequality-red.csv', 
    sep=';'
)
winedf.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Посмотрим, что в датасете нет пропущенных значений:

In [3]:
winedf['quality'].isnull().sum()

0

Разделим данные на регрессоров и отклик:

In [4]:
X = winedf.drop(columns=['quality'])
y = winedf['quality']

print(type(X), type(y)) 

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


## Использование `Pipeline`

Для создания пайплайна следует определить последовательность именованных шагов. В качестве примера построим пайплайн для логистической регрессии. Так как в модели есть регуляризация, то данные предварительно надо стандартизировать. Таким образом, получим шаги:
1. standard scaler;
2. логистическая регрессия. 

*Замечание.*

Кроме `StandardScaler` в `sklearn` есть еще набор классов для масштабирования:

* `MinMaxScaler`
* `MaxAbsScaler`
* `StandardScaler`
* `RobustScaler`
* `Normalizer`
* `QuantileTransformer`
* `PowerTransformer`

In [5]:
steps = [
    ('scaler', StandardScaler()), 
    ('clf', LogisticRegression(solver='saga', 
                               multi_class='multinomial', 
                               max_iter=5000))
]

pipeline = Pipeline(steps) 

Разделим выборку на обучающую и тестовую. Перед этим посмотрим на распределение классов у отклика.

In [6]:
y.value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

Распределение классов очень несбалансированное, поэтому будем использовать стратификацию при разделении на обучающую и тестовую выборки.

*Замечание.*

Подробнее о значении стратификации можете прочесть в ноутбуке про валидацию.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=30, stratify=y
)

Обучим созданный ранее `Pipeline` на обучающей выборке. Опишем на данном конкретном примере, что произойдет после вызова метода `fit`:
1. У `pipiline.named_steps['scaler']` вызовется метод `fit_transform` над `X_train`, который обучит скейлер и вернет преобразованные данные. Для упрощения дальнейшего изложения обозначим их `X_train_scaled`.
2. У `pipeline.named_steps['clf']` вызовется метод `fit` над `X_train_scaled`, так как это финальная компонента.

In [8]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=5000,
                                    multi_class='multinomial', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='saga', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

Выполним предсказания на тестовой выборке и измерим долю правильных ответов в предсказаниях. Опишем на данном конкретном примере, что произойдет после вызова метода `predict`:
1. У `pipiline.named_steps['scaler']` вызовется метод `transform` над `X_test`, который вернет преобразованные данные. Для упрощения дальнейшего изложения обозначим их `X_test_scaled`.
2. У `pipeline.named_steps['clf']` вызовется метод `predict` над `X_test_scaled`, так как это финальная компонента.

In [9]:
prediction = pipeline.predict(X_test)
accuracy = (prediction == y_test).mean()
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.584


## Подбор гиперпараметров при помощи `Pipeline`

Как мы уже выяснили, объект класса `Pipeline` может быть моделью машинного обучения, а потому мы можем захотеть определить для этой модели оптимальные гиперпараметры. 

Чтобы получить доступ к параметрам объектов внутри `named_steps` требуется обратиться к параметру с названием `{name_step}__{param_name}`, где `name_step` &mdash; имя шага, `param_name` &mdash; название параметра. 

Возвращаясь к нашему примеру, мы хотим подобрать оптимальные параметры у логистической регрессии. Обычно оптимизируют `penalty` и `C`. 

Но почему мы здесь хотим использовать именно `Pipeline`? Почему бы не обучить `StandardScaler` на всей обучающей выборке и не выполнить соответствующее преобразование? Как мы уже знаем, на каждом этапе кросс-валидации от обучающей выборки отрезается один фолд на валидацию, а остальные идут на обучение. Поэтому, если мы применим `StandardScaler` в самом начале ко всей выборке, то фолды обучения увидят некоторую информацию о фолде валидации. В общем же случае, фолды обучения не должны знать валидационные и тестовые данные. Таким образом, наша валидационная процедура будет отличаться от тестирования, что может приводить к плохой корреляции результатов валидации и тестирования, а также к завышенным результатам валидации.

In [10]:
parameteres = {'clf__penalty': ['l1', 'l2'], 
               'clf__C': [0.01, 0.1, 1, 10, 100]}

Теперь создадим объект для поиска по сетке `GridSearchCV`, которой передадим `Pipeline` типа `Predictor` и скажем, что будем использовать кросс-валидацию на $5$ фолдов:

In [11]:
grid = GridSearchCV(
    pipeline, param_grid=parameteres, cv=5, scoring='accuracy'
)

Выполним поиск:

In [12]:
%%time
grid.fit(X_train, y_train)

CPU times: user 35.3 s, sys: 5.06 ms, total: 35.3 s
Wall time: 35.4 s


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        LogisticRegression(C=1.0,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           l1_ratio=None,
                                                           max_iter=5000,
                                                           multi_class='multinomial',
     

Найдем качество лучшей найденной модели и выведем оптимальные гиперпараметры:

In [13]:
print(f"Accuracy = {grid.score(X_test, y_test):.2f}")
print(grid.best_params_)

Accuracy = 0.58
{'clf__C': 0.1, 'clf__penalty': 'l2'}


## Плюсы использования `Pipeline`

1. Создание пайплайнов обеспечивает соблюдение определенного порядка выполнения операций, что способствует воспроизводимости и созданию удобного рабочего процесса.
2. Полученный объект может быть передан во все процедуры библиотеки `sklearn` наравне со стандартными классами моделей. Например, это позволяет делать поиск гиперпараметров при помощи процедуры `GridSearchCV`.
3. Структура класса позволяет лучше избежать утечек данных с валидационных множеств во время кросс-валидации.